In [1]:
import Agent_Library
from   Agent_Library import np, plt, Camera

import pandas as pd

import importlib
# import psutil
# import cProfile, pstats
# import snakeviz.cli as cli
# %load_ext snakeviz

In [2]:
def Initializer(N, L):
    importlib.reload(Agent_Library)
    np.random.seed(41)
    # -----------------------------------------------------------------------------------
    camera = Camera(plt.figure(figsize=(15,15)))

    # Build the environment and the agents ----------------------------------------------
    Agents, states = [], []
    for _ in range(N): 
        Agents.append(Agent_Library.Distributed_Agent(L, flipping=True, moving=False, training=False))
        states.append([ 1, 1, 1 ])
    
    environment = Agent_Library.Plot_Environment(L, Agents)
    
    # Parameters displayed in plots -----------------------------------------------------
    Hamilton = []               # To display the Hamiltonian value per episede
    Giant  = []                 # To display the percentage of members of the giant component 
    Edges  = []                 # To display the number of links in the network
    Energy = []                 # To display the amount of energy consumed by all agents
    Tau    = []                 # To show the stability of links
    
    return camera, environment, Agents, states,  Hamilton, Giant, Edges, Energy, Tau

In [ ]:
N = 8                      # Number of agents
L = 6                       # The length of the simulation box
camera, environment, Agents, states, Hamilton, Giant, Edges, Energy, Tau = Initializer(N, L)


for episode in range(50):
    for i in range(len(Agents)):
        states[i] = Agents[i].Prediction(states[i], (Agents[:i] + Agents[i+1:]))

    # ---------------------------------------------------------------------------------------
    # delta_HD = []
    # for i in range(N-1):
    #     distance = ( (Agents[0].x - Agents[0].OtherAgents[i].x)**2 + (Agents[0].y-Agents[0].OtherAgents[i].y)**2 )**0.5
    #     if distance <= Agents[0].OtherAgents[i].r and distance > Agents[0].r:
    #         delta_H = -0.5*((Agents[0].k+1)**2 - Agents[0].k**2) + 0.1*((Agents[0].k+1)**3 - Agents[0].k**3) + 0.2*(distance**2 - Agents[0].r**2) - 0.5*(1/distance)
    #         delta_HD.append([ delta_H, distance ])
    #         if np.exp(-delta_HD[np.argmin(np.array(delta_HD)[:,0])][0] / 4) > np.random.random():
    #             Agents[0].r = delta_HD[np.argmin(np.array(delta_HD)[:,0])][1]
    #             print("\n", i+1, delta_HD[np.argmin(np.array(delta_HD)[:,0])][1], delta_HD)

    # Show Results --------------------------------------------------------------------------
    hamilton, edge, energy, giant, tau = environment.Calculate_Result(Agents, episode)
    Hamilton.append(hamilton)
    Edges.append(edge)
    Energy.append(energy)
    Giant.append(giant)
    Tau.append(tau)

    if episode%1 == 0: environment.Animation(camera, episode)

    # cpu_percent = psutil.cpu_percent(interval=1, percpu=True)
    print("\rEpisode: {}, H: {:.3f}, Giant: {:.2f}, N: {:.1f}".format(episode, hamilton, giant, environment.N), end="")

anim = camera.animate(interval= 420, repeat=True, repeat_delay= 500, blit=True)
anim.save('./All result/animation.gif')

#-------------------------------------------------------------------------------
print("\n\n", np.average(Hamilton))
environment.Static_Plot(episode, Hamilton, Edges, Energy, Giant, Tau[10:])

#-------------------------------------------------------------------------------
df = pd.DataFrame({'Hamilton': Hamilton, 'Giant': Giant, 'Edges': Edges, 'Energy': Energy, 'Tau': Tau})
df.to_csv('./All result/csv/Results.csv')



In [ ]:
# with cProfile.Profile() as profile: main()
# stats = pstats.Stats(profile).sort_stats('cumtime')
# stats.dump_stats("./All result/cProFile/cProFile.prof")

# stats.print_stats()
# cli.main(["./All result/cProFile/cProFile.prof"])